# Valorización de derivados SUC en Python

In [1]:
import pyodbc
import pandas as pd
import numpy as np
import datetime
from Derivados.DerivadosSUC import *

## Datos por defecto

In [2]:
info_derivado = dict()
info_derivado["Tipo"] = 'SUC'
info_derivado["ID_Key"] = ""
info_derivado["Administradora"] = "Banco de Chile"
info_derivado["Fondo"] = "Fondo"
info_derivado["Contraparte"] = "Banco Penta"
info_derivado["ID"] = "id"
info_derivado["Nemotecnico"] = ""
info_derivado["Mercado"] = "Local"


## Conexión a base de datos

In [3]:
import ConfiguracionConexiones

conexiones = ConfiguracionConexiones.Conexiones_produccion()

cnn = conexiones.get_input()

## Datos para valorizar

In [4]:
fecha = datetime.date(2020, 1, 2)
hora = '1700'
info_derivado["ID"] = 5
info_derivado["FechaEfectiva"] = "2/1/2019"
info_derivado["FechaVenc"] = "2/1/2021"

info_derivado["AjusteFeriados"] = "CL"

info_derivado["NocionalActivo"] = 100000

info_derivado["MonedaActivo"] = 'UF'
info_derivado["MonedaBase"] = 'CLP'

info_derivado["TipoTasaActivo"] = 'Fija'
info_derivado["TipoTasaPasivo"] = 'Flotante'

info_derivado["TasaActivo"] = 0.72
info_derivado["TasaPasivo"] = -1000

info_derivado["FrecuenciaActivo"] = "Semi annual"
info_derivado["FrecuenciaPasivo"] = info_derivado["FrecuenciaActivo"]


info = pd.DataFrame([info_derivado])

In [5]:
derivado = DerivadosSUC(fecha, hora, info, cnn)

In [6]:
derivado.get_status()

'INFO: Derivado inicializado'

In [7]:
derivado.genera_flujos()


AQUI HAY ALGO RARO
[datetime.date(2020, 7, 2), -0.7831832494438862, datetime.date(2020, 7, 2), 0, -0.7831832494438862, 0.005094453997001036, 0.0]
AQUI HAY ALGO RARO
[datetime.date(2021, 1, 2), 99.52289035882312, datetime.date(2021, 1, 4), 100, -0.47710964117688714, 0.0051896563336151225, 0]


In [8]:
derivado.flujos_derivados

,Fecha,Fondo,Tipo,ID,ActivoPasivo,FechaFixing,FechaFlujo,FechaPago,Moneda,Flujo,Amortizacion,Interes,Sensibilidad,InteresDevengado
0,2020-01-02,Fondo,SUC,5,-1,2020-07-02,2020-07-02,2020-07-02,UF,-783.183249,0.0,-783.183249,5.094454,0.0
0,2020-01-02,Fondo,SUC,5,-1,2021-01-02,2021-01-02,2021-01-04,UF,99522.890359,100000.0,-477.109641,5.189656,0.0
0,2020-01-02,Fondo,SUC,5,1,2020-07-02,2020-07-02,2020-07-02,UF,364.000000,0.0,364.000000,0.000000,0.0
0,2020-01-02,Fondo,SUC,5,1,2021-01-02,2021-01-02,2021-01-04,UF,100368.000000,100000.0,368.000000,0.000000,0.0


In [9]:
derivado.get_status()

'INFO: Flujos generados para SUC con ID 5'

In [10]:
derivado.valoriza_flujos()

SELECT Curva, FechaMax, C.Hora, C.Moneda FROM dbDerivados.dbo.TdCurvasDerivados A, dbDerivados.dbo.TdParidadMonedasCurvasDescuento B, (SELECT MAX(Fecha) AS FechaMax, Hora, Moneda FROM dbDerivados.dbo.TdCurvasDerivados A, dbDerivados.dbo.TdParidadMonedasCurvasDescuento B WHERE A.Tipo = B.Tipo AND Fecha <= '20200102' AND Hora = '1700' AND Moneda IN ('CLF','CLP') GROUP BY Hora, Moneda) C WHERE A.Tipo = B.Tipo AND A.Fecha = C.FechaMax AND A.Hora = C.Hora AND B.Moneda = C.Moneda


In [11]:
derivado.get_status()

'INFO: Valorización completada con éxito'

In [12]:
derivado.agrega_cambio_spot()

/home/mvillegas/.local/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


# Flujos valorizados

In [13]:
derivado.flujos_valorizados[["TipoCambioSpot", "TipoCambioFwd", "TipoCambioSpotFix", "TipoCambioCLPBase"]]

,TipoCambioSpot,TipoCambioFwd,TipoCambioSpotFix,TipoCambioCLPBase
0,28311.77,28734.611867,28311.77,1.0
1,28311.77,29149.196001,28311.77,1.0
2,28311.77,28734.611867,28311.77,1.0
3,28311.77,29149.196001,28311.77,1.0


## Valorización completa

In [14]:
infomio = derivado.flujos_valorizados

print("Activo")
act = sum(infomio[infomio["ActivoPasivo"] == 1].ValorPresenteCLP)
print("Valor activo:", act)
print()
print("Pasivo")
pas = sum(infomio[infomio["ActivoPasivo"] == -1].ValorPresenteCLP)
print("Valor pasivo:", pas)


print(act-pas)

Activo
Valor activo: 2887853550.4715962

Pasivo
Valor pasivo: 2830889798.3835645
56963752.08803177


In [15]:
5.427235740612841E7

54272357.40612841